In [8]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel




In [9]:
df = pd.read_csv("books.csv", on_bad_lines='skip')
display(df.head(5))
display(df.describe())
display(df.info())
display(display(print (df.isna().sum())))

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,9/16/2006,Scholastic Inc.
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,9/1/2004,Scholastic Inc.
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,0439554896,9780439554893,eng,352,6333,244,11/1/2003,Scholastic
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,9780439655484,eng,435,2339585,36325,5/1/2004,Scholastic Inc.
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,0439682584,9780439682589,eng,2690,41428,164,9/13/2004,Scholastic


,bookID,average_rating,isbn13,num_pages,ratings_count,text_reviews_count
count,11123.000000,11123.000000,1.112300e+04,11123.000000,1.112300e+04,11123.000000
mean,21310.856963,3.934075,9.759880e+12,336.405556,1.794285e+04,542.048099
std,13094.727252,0.350485,4.429758e+11,241.152626,1.124992e+05,2576.619589
min,1.000000,0.000000,8.987060e+09,0.000000,0.000000e+00,0.000000
25%,10277.500000,3.770000,9.780345e+12,192.000000,1.040000e+02,9.000000
50%,20287.000000,3.960000,9.780582e+12,299.000000,7.450000e+02,47.000000
75%,32104.500000,4.140000,9.780872e+12,416.000000,5.000500e+03,238.000000
max,45641.000000,5.000000,9.790008e+12,6576.000000,4.597666e+06,94265.000000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11123 entries, 0 to 11122
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   bookID              11123 non-null  int64  
 1   title               11123 non-null  object 
 2   authors             11123 non-null  object 
 3   average_rating      11123 non-null  float64
 4   isbn                11123 non-null  object 
 5   isbn13              11123 non-null  int64  
 6   language_code       11123 non-null  object 
 7     num_pages         11123 non-null  int64  
 8   ratings_count       11123 non-null  int64  
 9   text_reviews_count  11123 non-null  int64  
 10  publication_date    11123 non-null  object 
 11  publisher           11123 non-null  object 
dtypes: float64(1), int64(5), object(6)
memory usage: 1.0+ MB


None

bookID                0
title                 0
authors               0
average_rating        0
isbn                  0
isbn13                0
language_code         0
  num_pages           0
ratings_count         0
text_reviews_count    0
publication_date      0
publisher             0
dtype: int64


None

None

# Popularity based recommender

In [10]:
# Define the formula for the popilarit ybased recommendation
df_pop = df.copy()
df_pop['new_rating'] = (df_pop['ratings_count']/df_pop['ratings_count'].max())*df_pop['average_rating']

In [13]:
df_pop.sort_values(by=['new_rating'], inplace=True, ascending=False)
recommended_books = df_pop.iloc[:5,:]
display(recommended_books)

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,new_rating
10336,41865,Twilight (Twilight #1),Stephenie Meyer,3.59,0316015849,9780316015844,eng,501,4597666,94265,9/6/2006,Little Brown and Company,3.590000
1697,5907,The Hobbit or There and Back Again,J.R.R. Tolkien,4.27,0618260307,9780618260300,eng,366,2530894,32871,8/15/2002,Houghton Mifflin,2.350522
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,9780439655484,eng,435,2339585,36325,5/1/2004,Scholastic Inc.,2.320418
4415,15881,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling/Mary GrandPré,4.42,0439064864,9780439064866,eng,341,2293963,34692,6/2/1999,Arthur A. Levine Books / Scholastic Inc.,2.205318
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,9/1/2004,Scholastic Inc.,2.102745


# Content-based Recommender

In [75]:
def ContentBasedRecommender(df, author, indices, distance_matrix):
    id_ = indices[author]
    distances = list(enumerate(distance_matrix[id_]))
    recommendations = [distance[0] for distance in distances]
    
    return df[['authors','title']].iloc[recommendations]

In [79]:
df = pd.read_csv("books.csv", on_bad_lines='skip')

#Instantiate a new Vectorizer object 
tfidf = TfidfVectorizer(stop_words='english')

#Replace NaN with an empty string
df['authors'] = df['authors'].fillna('')

#transform the author of the book to the tfidf matrix needed
tfidf_matrix = tfidf.fit_transform(df['authors'])

# use the cosine_similarity for the tfidf matrix
distance_matrix = linear_kernel(tfidf_matrix)
indices = pd.Series(df.index, index=df['authors']).drop_duplicates()
#display(indices)

output = ContentBasedRecommender(df, "J.K. Rowling/Mary GrandPré", indices, distance_matrix)
output

,authors,title
0,J.K. Rowling/Mary GrandPré,Harry Potter and the Half-Blood Prince (Harry ...
1,J.K. Rowling/Mary GrandPré,Harry Potter and the Order of the Phoenix (Har...
2,J.K. Rowling,Harry Potter and the Chamber of Secrets (Harry...
3,J.K. Rowling/Mary GrandPré,Harry Potter and the Prisoner of Azkaban (Harr...
4,J.K. Rowling/Mary GrandPré,Harry Potter Boxed Set Books 1-5 (Harry Potte...
5,W. Frederick Zimmerman,"Unauthorized Harry Potter Book Seven News: ""Ha..."
